#**Images and Words**
This project pretends to use common Deep Learning techniques more commonly used on the fields of image and/or speech in order to improve current methods of music generation.

<b>Rember to set up Python 3 with GPU!!!!!!</b>

<a href='https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5'>Link</a> to the original article where I found the whole thing. It is interesting specially when it comes to the data pre-traitment. I think it is a little bit amateur... (Base on some of the coding)

Introducing the <a href='https://github.com/bearpelican/music_rnn/blob/master/bachbot/BachBot-StackedLSTMs.ipynb' >bearpelican</a> pytorch implementation:


In [0]:
#Clone the original git, just for the dataset (I think is base on Final Fantasy scores...)
!git clone https://github.com/Skuldur/Classical-Piano-Composer.git
#It also helps with the directories


##*Libraries*
They should run correctly in colab and in local as long as you use Python 3.

The key library is music21, central to the whole pre-traitement of the midi dataset. It has been developed by the MIT and you can find here the <a href='http://web.mit.edu/music21/doc/index.html'> documentation</a>.

.


In [0]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential #TOBE Kill
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
import torch
import torch.nn 
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.optim as optim


#*Pre-Traitment of Dataset*

##*1. Midi to Text*


In [0]:

#You should adapt the directories. The way they are, they are think to be run directly from Colab

def get_notes(n=-1):
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for i,file in enumerate(glob.glob("/content/Classical-Piano-Composer/midi_songs/*.mid")):
        
        if n==i:
            break  
        
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/Classical-Piano-Composer/data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [0]:
#convert the whole data set to text (the separation between train and test is made later I think...)

#Idea Martí:------> We should adapt get_notes() so that you can select a subset of midis
trainset_notes=get_notes(5)

Parsing /content/Classical-Piano-Composer/midi_songs/Zelda_Overworld.mid
Parsing /content/Classical-Piano-Composer/midi_songs/OTD5YA.mid
Parsing /content/Classical-Piano-Composer/midi_songs/FFIXQuMarshP.mid
Parsing /content/Classical-Piano-Composer/midi_songs/ahead_on_our_way_piano.mid
Parsing /content/Classical-Piano-Composer/midi_songs/Ff4-BattleLust.mid


In [0]:
len(trainset_notes)

3466

In [0]:
midi = converter.parse("/content/Classical-Piano-Composer/midi_songs/Ff7-One_Winged.mid")
s2 = instrument.partitionByInstrument(midi)

In [0]:
midi[1][1]

<music21.key.Key of G major>

In [0]:
for i in midi.flat.notes[6:30]:
  if isinstance(i, note.Note):
    print("{} {}".format(i.pitch,i.quarterLength)) 
  else:
     print("{} {}".format(i,i.duration))

<music21.chord.Chord A3 B-3 E4> <music21.duration.Duration 0.75>
E3 0.75
<music21.chord.Chord A3 B-3 E4> <music21.duration.Duration 0.75>
E2 0.75
<music21.chord.Chord A3 B-3 E4> <music21.duration.Duration 0.75>
E3 0.75
<music21.chord.Chord A3 B-3 E4> <music21.duration.Duration 0.75>
E2 0.75
<music21.chord.Chord A3 B-3 E4> <music21.duration.Duration 0.75>
E3 0.75
E-6 0.0
E6 1/3
E-5 0.5
A5 0.5
E5 0.5
E-6 0.0
E6 1/3
F5 0.25
E5 0.25
A5 0.5
E-5 0.5
E-6 0.0
E6 1/3
E5 0.5


In [0]:
s2.parts[0].recurse()[6].quarterLength

0.75

In [0]:
len(midi[1])

869

In [0]:
trainset_notes[0]

'B-4'

##*2. Prepare Inputs and Outputs*

Separate the whole dataset into consecutive sequeces

In [0]:
#prepare the data set in a way that is useful for the Network


def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 25

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output,int_to_note)

In [0]:
# get amount of pitch names (Final dimension of the softmax)
n_vocab = len(set(trainset_notes))

#Martí:--> Not sure where separetes train and test ... (i think it just uses all)    
network_input, network_output,int_to_note = prepare_sequences(trainset_notes, n_vocab)

In [0]:

print(len(int_to_note))
print(network_input.shape)
print(network_output.shape)


113
(3441, 25, 1)
(3441, 113)


In [0]:
int_to_note


{0: '0',
 1: '0.2',
 2: '0.4',
 3: '0.4.7',
 4: '0.5',
 5: '1.3.6',
 6: '1.4.8',
 7: '1.5.8',
 8: '1.6',
 9: '10.1',
 10: '11',
 11: '11.0',
 12: '11.0.5',
 13: '11.1.4',
 14: '11.2',
 15: '11.2.5',
 16: '11.3',
 17: '11.3.6',
 18: '11.4',
 19: '2',
 20: '2.4.5',
 21: '2.5',
 22: '2.5.9',
 23: '2.7',
 24: '2.8',
 25: '3.6',
 26: '3.8',
 27: '4.5',
 28: '4.6',
 29: '4.7',
 30: '4.9',
 31: '5',
 32: '5.11',
 33: '5.7',
 34: '5.9',
 35: '5.9.0',
 36: '6.10',
 37: '6.10.1',
 38: '6.11',
 39: '7',
 40: '7.0',
 41: '7.11',
 42: '7.11.2',
 43: '7.9',
 44: '8.1',
 45: '8.11',
 46: '8.11.1',
 47: '9',
 48: '9.0',
 49: '9.0.2',
 50: '9.0.2.5',
 51: '9.1',
 52: '9.10.2.4',
 53: '9.11',
 54: '9.11.2',
 55: '9.2',
 56: 'A1',
 57: 'A2',
 58: 'A3',
 59: 'A4',
 60: 'A5',
 61: 'B-2',
 62: 'B-3',
 63: 'B-4',
 64: 'B-5',
 65: 'B1',
 66: 'B2',
 67: 'B3',
 68: 'B4',
 69: 'C#1',
 70: 'C#2',
 71: 'C#3',
 72: 'C#4',
 73: 'C#5',
 74: 'C#6',
 75: 'C2',
 76: 'C3',
 77: 'C4',
 78: 'C5',
 79: 'C6',
 80: 'D1',
 81:

#*Network and Training in Pytorch (or Keras?)*

In [0]:
#Adapt the Keras into Alice in the WonderLand
import torch.nn as nn



#KERAS!!!!!!!!!!!!!!!!!!!!!!!
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model
  
#KERAS!!!!!!!!!!!!!!!!!!!!!!!
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=5, batch_size=64, callbacks=callbacks_list)

In [0]:
model = create_network(network_input, n_vocab)

train(model, network_input, network_output)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
4189/4189 [==============================] - 6s 1ms/step - loss: 4.5178
Epoch 2/5
4189/4189 [==============================] - 1s 329us/step - loss: 4.4035
Epoch 3/5
4189/4189 [==============================] - 1s 334us/step - loss: 4.3351
Epoch 4/5
4189/4189 [==============================] - 1s 335us/step - loss: 4.3036
Epoch 5/5
4189/4189 [==============================] - 1s 330us/step - loss: 4.2774


In [0]:
network_input.shape[2]

1

#*Prediction in Pytorch*

In [0]:
network_input.shape

(4189, 3, 1)

In [0]:
def generate_notes(model, network_input, int_to_note, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    #int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(20):
        
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))

#        prediction_input = prediction_input #/ float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
       
        #pattern.append(index)
        pattern=numpy.append(pattern,index/ float(n_vocab))
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [0]:
prediction_output = generate_notes(model, network_input, int_to_note, n_vocab)


In [0]:
prediction_output

['C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3',
 'C3']

In [0]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [0]:
#model.load_weights('/content/Classical-Piano-Composer/new_weights.hdf5')
#int_to_note


In [0]:
create_midi(prediction_output)

In [0]:
n_vocab

163

In [0]:
import torch
from torch.utils import data
import numpy as np

class MidiDataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, seq_len=25):
        'Initialization'
        self.notes = self.get_notes(n=5)
        self.seq_len = seq_len
        
        # get all pitch names
        self.pitchnames = sorted(set(item for item in self.notes))

        # create a dictionary to map pitches to integers
        self.note_to_int = dict((note, number) for number, note in enumerate(self.pitchnames))
        self.int_to_note = dict((number, note) for number, note in enumerate(self.pitchnames))
        self.vocab_size = len(self.pitchnames)
        

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.notes) - self.seq_len*2

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        samples = self.notes[index:(index+self.seq_len*2)]
        x = np.zeros((self.seq_len*2, self.vocab_size)).astype(np.float32)
        for i,s in enumerate(samples):
            j = self.note_to_int[s]
            x[i,j] = 1.0

        return  (x[0:self.seq_len,:] , x[self.seq_len:,:])
      
  def get_notes(self, n=-1):
      """ Get all the notes and chords from the midi files in the ./midi_songs directory """
      notes = []

      for i,file in enumerate(glob.glob("/content/Classical-Piano-Composer/midi_songs/*.mid")):

          if n==i:
              break  

          midi = converter.parse(file)

          print("Parsing %s" % file)

          notes_to_parse = None

          try: # file has instrument parts
              s2 = instrument.partitionByInstrument(midi)
              notes_to_parse = s2.parts[0].recurse() 
          except: # file has notes in a flat structure
              notes_to_parse = midi.flat.notes

          for element in notes_to_parse:
              if isinstance(element, note.Note):
                  notes.append(str(element.pitch))
              elif isinstance(element, chord.Chord):
                  notes.append('.'.join(str(n) for n in element.normalOrder))

      return notes

In [0]:
training_generator = data.DataLoader(MidiDataset(seq_len=13), batch_size=23, shuffle=True)

Parsing /content/Classical-Piano-Composer/midi_songs/Zelda_Overworld.mid
Parsing /content/Classical-Piano-Composer/midi_songs/OTD5YA.mid
Parsing /content/Classical-Piano-Composer/midi_songs/FFIXQuMarshP.mid
Parsing /content/Classical-Piano-Composer/midi_songs/ahead_on_our_way_piano.mid
Parsing /content/Classical-Piano-Composer/midi_songs/Ff4-BattleLust.mid


In [0]:
for batch in training_generator:
    break
print(batch[0].shape)

torch.Size([23, 13, 163])


In [0]:
import torch.nn as nn

class NextNoteModel(nn.Module):
  def __init__(self, input_dim, rnn_dim=512, rnn_layers=2):
      super(NextNoteModel, self).__init__()
      self.rnn = nn.LSTM( input_size=input_dim, hidden_size=rnn_dim, num_layers=rnn_layers, batch_first=True, dropout=0.2)
      self.classifier = nn.Sequential(
          nn.Linear(rnn_dim, 256),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(256, input_dim)
      )
      self.loss_function = nn.CrossEntropyLoss() #combina logsoftmax y NLLLoss
      
  def forward(self, x):          
#           output, (hn, cn) = rnn(input, (h0, c0))
      output, (hn, cn) = self.rnn(x)
      return self.classifier(output[:,-1,:]) #no hace falta la softmax
    
  def loss(self, x,y):
      y_pred = y.argmax(dim=1)
      return self.loss_function(x,y_pred)
    
  
  def accuracy(self, x, y):
      x_pred = x.argmax(dim=1)
      y_pred = y.argmax(dim=1)
      return (x_pred == y_pred).float().mean()
    
      
      

In [0]:
import torch.nn as nn
https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb
  
class Seq2Seq(nn.Module):
  def __init__(self, input_dim, rnn_dim=512, rnn_layers=2):
      super(NextNoteModel, self).__init__()
      self.encoder = nn.LSTM( input_size=input_dim, hidden_size=rnn_dim, num_layers=rnn_layers, batch_first=True, dropout=0.2)
      self.decoder = nn.LSTM( input_size=input_dim, hidden_size=rnn_dim, num_layers=rnn_layers, batch_first=True, dropout=0.2)
      self.classifier = nn.Sequential(
          nn.Linear(rnn_dim, 256),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(256, input_dim)
      )
      self.loss_function = nn.CrossEntropyLoss() #combina logsoftmax y NLLLoss
      
  def forward(self, x,y):          
#           output, (hn, cn) = rnn(input, (h0, c0))
      output, (hn, cn) = self.encoder(x)
      y = torch.cat([torch.zeros( (y.shape[0],1,y.shape[2]),) , y] , dim=1)  
      output, (hn, cn) = self.decoder(y, (hn,cn)) #el y tiene que tener paddding de cero, por ejemplo para marcar el inicio de sequencia
      
      shape = output.shape
      x= output.view(shape[0]*shape[1], -1)
      
      x = self.classifier(x) #no hace falta la softmax
      x = x.view(shape[0,shape[1],-1])
      return x
    
  def loss(self, x,y):
    
      #flatten 
      y_pred = y.argmax(dim=1)
      return self.loss_function(x,y_pred)
    
  
  def accuracy(self, x, y):
      #flatten
      x_pred = x.argmax(dim=1)
      y_pred = y.argmax(dim=1)
      return (x_pred == y_pred).float().mean()
    
      
      

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model = NextNoteModel(163)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)




In [0]:

out = modelo(batch[0])

torch.Size([23, 13, 512])


In [0]:
batch.shape

torch.Size([23, 13, 163])

In [0]:
print(training_generator.dataset.vocab_size)
training_generator.dataset.int_to_note[5]

163


'0.3.6.8'

In [0]:
for epoch in range(200):
    for i,batch in enumerate(training_generator):
        # Forward pass: Compute predicted y by passing x to the model
        x = batch[0]
        y = batch[1][:,0,:]
        x= x.to(device)
        y= y.to(device)
        y_pred = model(x)

        # Compute and print loss
        loss = model.loss(y_pred, y)
        acc  = model.accuracy(y_pred, y)


        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(epoch, loss.item(), acc.item())
  
  


0 0.007250308990478516 1.0
1 0.026912927627563477 1.0
2 0.00890620518475771 1.0
3 0.06554309278726578 1.0
4 0.3337826728820801 0.6666666865348816
5 0.8159777522087097 0.6666666865348816
6 0.19464564323425293 1.0
7 0.025013843551278114 1.0
8 0.026012739166617393 1.0
9 0.009528915397822857 1.0
10 0.06173849105834961 1.0
11 0.4664026200771332 0.6666666865348816
12 0.011328518390655518 1.0
13 0.5870955586433411 0.6666666865348816
14 0.01793956756591797 1.0
15 0.0011755625018849969 1.0
16 0.006022373680025339 1.0
17 0.003963311668485403 1.0
18 0.021434267982840538 1.0
19 0.043759655207395554 1.0
20 0.04441845417022705 1.0
21 0.07979337126016617 1.0
22 0.766578197479248 0.6666666865348816
23 0.04107789322733879 1.0
24 0.010071913711726665 1.0
25 0.03142392635345459 1.0
26 0.021651705726981163 1.0
27 0.028817078098654747 1.0
28 0.05566573143005371 1.0
29 0.010359048843383789 1.0
30 0.0014175176620483398 1.0
31 0.6774728894233704 0.6666666865348816
32 0.12093877792358398 1.0
33 0.0166737530380

KeyboardInterrupt: ignored

In [0]:
testing_generator = data.DataLoader(MidiDataset(seq_len=13), batch_size=1, shuffle=False)

for batch in testing_generator:
    x = batch[0]
    y = batch[1][:,0,:]
    x= x.to(device)
    y= y.to(device)
    y_pred = model(x)
    y_pred = y_pred.argmax(dim=1)
    y_pred = y_pred.cpu().numpy()
    print(testing_generator.dataset.int_to_note[y_pred[0]])
    

Parsing /content/Classical-Piano-Composer/midi_songs/Zelda_Overworld.mid
Parsing /content/Classical-Piano-Composer/midi_songs/OTD5YA.mid
Parsing /content/Classical-Piano-Composer/midi_songs/FFIXQuMarshP.mid
Parsing /content/Classical-Piano-Composer/midi_songs/ahead_on_our_way_piano.mid
Parsing /content/Classical-Piano-Composer/midi_songs/Ff4-BattleLust.mid
C4
B-4
C4
B-4
B-4
C4
B-4
C4
B-4
C4
B-4
C#4
G#4
C#4
B-4
C#4
B-4
C#4
B-4
C#4
B-4
C#4
B-4
C#4
B-4
C#4
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
D4
F4
D4
D4
C4
D4
B-4
D4
B-4
D4
C5
E-4
D5
F4
E-5
G4
F5
G#4
B-4
B-4
C5
D5
E-5
F5
F5
F5
F5
G#4
F#5
B-4
G#5
C5
B-5
C#5
C#6
F#4
G#4
B-4
C5
C#5
B-5
C#5
B-5
B-5
C#5
G#5
C5
F#5
B-4
G#5
C#5
F#5
G#4
F5
G#4
G#4
F#4
G#4
G#4
F5
G#4
F#4
G#4
E-5
F#4
E-5
E-5
F5
F5
F#5
E-5
F#4
G#4
B-4
F5
G#4
E-5
F#4
C#5
F4
C#5
F4
E-5
E-4
F5
F4
F4
F#4
G#4
E-5
F#4
C#5
F4
C5
E4
C5
D5
E5
E4
F4
G4
G4
A4
G5
B-4
C5
F5
A4
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
A3
F4
D4
F4
D4
D4
C4
D4
B-4
D4
B-4
D4

KeyboardInterrupt: ignored